In [10]:
import pandas as pd
from glob import glob
from os import path

import sys
sys.path.append(path.abspath(".."))

from src.portfolio_tools import compute_equal_weights, compute_vol_scaled_weights, apply_weights

files = glob("../data/processed/*.parquet")
monthly_returns = {
    path.splitext(path.basename(f))[0].replace("monthly_returns_", ""): pd.read_parquet(f)
    for f in files
}

monthly_returns

{'2022-02':             ARPAUSDT  AUDIOUSDT   ICXUSDT   ENSUSDT  BTCDOMUSDT  IOSTUSDT  \
 close_time                                                                  
 2022-02-01  0.002749   0.069289 -0.005713 -0.039474   -0.024195 -0.025894   
 2022-02-02 -0.033710  -0.081712 -0.054650 -0.072177   -0.002877 -0.029167   
 2022-02-03  0.028710   0.015422  0.035477  0.023204    0.001273  0.037866   
 2022-02-04  0.074477   0.114292  0.105788  0.120785    0.014324  0.074251   
 2022-02-05  0.040622   0.000196  0.021355 -0.028143   -0.014206  0.020508   
 2022-02-06  0.030039   0.090490  0.053179  0.039643    0.004662  0.018217   
 2022-02-07  0.284024   0.026522  0.039247  0.046679   -0.007341  0.045181   
 2022-02-08 -0.050472  -0.049396 -0.043617 -0.078091    0.020569 -0.017651   
 2022-02-09  0.000924   0.063203  0.006581  0.057632   -0.022320  0.016985   
 2022-02-10  0.200877   0.034229 -0.056894 -0.109642    0.028537 -0.062804   
 2022-02-11 -0.167179  -0.093255 -0.073097 -0.090323 

In [4]:
# ---- Equal-weight portfolio ----
portfolio_equal = []

months = sorted(monthly_returns.keys())
for i, month in enumerate(months):
    returns = monthly_returns[month]
    
    if returns.empty:
        continue
    
    # Equal weights
    weights = compute_equal_weights(returns)
    
    # Apply weights
    port_rets = apply_weights(returns, weights)
    portfolio_equal.append(port_rets)

portfolio_equal = pd.concat(portfolio_equal)
portfolio_equal

close_time
2020-01-02   -0.034067
2020-01-03    0.082274
2020-01-04    0.003125
2020-01-05    0.000612
2020-01-06    0.071785
                ...   
2025-04-26    0.030061
2025-04-27   -0.043630
2025-04-28    0.035095
2025-04-29   -0.024797
2025-04-30    0.030764
Length: 1946, dtype: float64

In [5]:
# ---- Volatility-scaled portfolio ----
portfolio_vol = []

for i in range(1, len(months)):
    prev_month = monthly_returns[months[i-1]]
    this_month = monthly_returns[months[i]]
    
    if this_month.empty or prev_month.empty:
        continue
    
    weights = compute_vol_scaled_weights(prev_month)
    port_rets = apply_weights(this_month, weights)
    portfolio_vol.append(port_rets)

portfolio_vol = pd.concat(portfolio_vol)
portfolio_vol

close_time
2020-02-01    0.011298
2020-02-02    0.005205
2020-02-03    0.003870
2020-02-04   -0.008382
2020-02-05    0.080718
                ...   
2025-04-26    0.019762
2025-04-27   -0.033151
2025-04-28    0.024468
2025-04-29   -0.018134
2025-04-30    0.020280
Length: 1916, dtype: float64